In [53]:
# open the file input.txt
with open('input.txt', 'r') as file:
    data = file.read().strip()

# split the data into lines
lines = data.split('\n')

# convert into a list of lists
maze = [list(line) for line in lines]

# find the start point demarked 'S'
x0 = None
for i, row in enumerate(maze):
    for j, cell in enumerate(row):
        if cell == 'S':
            x0 = (i, j)
            break

v0 = (0, 1)

In [54]:
def rotate_counterclockwise(v):
    return (-v[1], v[0])


def get_neighbours(maze, current_node):
    x, v = current_node

    neighbours = []
    
    for i in range(4):
        new_pos = (x[0] + v[0], x[1] + v[1])

        weight = min(i, 4 - i)*1000 + 1
        if maze[new_pos[0]][new_pos[1]] != '#':
            neighbours.append(((new_pos, v), weight))

        v = rotate_counterclockwise(v)

    return neighbours


In [56]:
# Vanilla Dijkstra's algorithm
def dijkstra_one_path(maze, x0, v0):
    shortest_path = {(x0, v0): (None, 0)}
    current_node = (x0, v0)
    visited = set()

    while maze[current_node[0][0]][current_node[0][1]] != 'E':
        visited.add(current_node)

        # get the neighbours
        neighbors = get_neighbours(maze, current_node)

        #cost of current node
        cost = shortest_path[current_node][1]

        for new_node, weight in neighbors:
            new_cost = cost + weight
            if new_node not in shortest_path:
                shortest_path[new_node] = (current_node, new_cost)
            else:
                current_shortest_cost = shortest_path[new_node][1]
                if new_cost < current_shortest_cost:
                    shortest_path[new_node] = (current_node, new_cost)

        next_destinations = {node: shortest_path[node] for node in shortest_path if node not in visited}

        if not next_destinations:
            return "No path found"
        
        current_node = min(next_destinations, key=lambda x: next_destinations[x][1])

    total_cost = shortest_path[current_node][1]
    
    # I don't need this, but ok
    path = []
    while current_node is not None:
        path.append(current_node)
        next_node = shortest_path[current_node][0]
        current_node = next_node
    # Reverse path
    path = path[::-1]

    return total_cost, path




In [57]:
cost, _ = dijkstra_one_path(maze, x0, v0)
print(cost)

104516


# Part 2

In [58]:
def dijkstra_all_paths(maze, x0, v0):
    shortest_paths = {(x0, v0): (None, 0)}  # this now stores a list of previous nodes
    current_node = (x0, v0)
    visited = set()

    while maze[current_node[0][0]][current_node[0][1]] != 'E':
        visited.add(current_node)

        # get the neighbours
        neighbors = get_neighbours(maze, current_node)

        #cost of current node
        cost = shortest_paths[current_node][1]

        for new_node, weight in neighbors:
            new_cost = cost + weight
            if new_node not in shortest_paths:
                shortest_paths[new_node] = ([current_node], new_cost)
            else:
                current_shortest_cost = shortest_paths[new_node][1]
                if new_cost < current_shortest_cost:
                    shortest_paths[new_node] = ([current_node], new_cost)
                elif new_cost == current_shortest_cost:  # add the current node to the list of previous nodes, all having the same value
                    shortest_paths[new_node][0].append(current_node)

        # explore the frontier
        next_destinations = {node: shortest_paths[node] for node in shortest_paths if node not in visited}

        if not next_destinations:
            return "No path found"
        
        current_node = min(next_destinations, key=lambda x: next_destinations[x][1])
    
    # get all nodes in all paths
    path_stack = [current_node]
    node_list = set()
    while path_stack:
        current_node = path_stack.pop()

        node_list.add(current_node)
        next_nodes = shortest_paths[current_node][0]

        if next_nodes:
            for node in next_nodes:
                if node not in node_list:
                    path_stack.append(node)

    return node_list


In [59]:
node_list = dijkstra_all_paths(maze, x0, v0)

# get the unique positions ignoring the directions
num_visited = len(set([pos for pos, _ in node_list]))
print(num_visited)



545
